# GPT vs GPT Chating, based on chatgpt assistant api

ChatGPT assist api example,  two GPT talks with each other.
After 5 message turns, it will finish.

Open it from [Colab](https://colab.research.google.com/github/davideuler/awesome-assistant-api/blob/main/GPT-VS-GPT.ipynb)

Reference:
https://github.com/yoheinakajima/GPTvsGPT/

In [4]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.9/219.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [3]:
import os
os.environ['OPENAI_API_KEY'] = "sk-xxx"

In [5]:
import time
import threading
import os
import openai
from openai import OpenAI

client = OpenAI()
client.api_key = os.environ.get('OPENAI_API_KEY')

def get_last_assistant_message(thread_id):
    messages_response = client.beta.threads.messages.list(thread_id=thread_id)
    messages = messages_response.data

    # Iterate through messages in reverse chronological order to find the last assistant message
    for message in messages:
        if message.role == 'assistant':
            # Get the content of the last assistant message
            assistant_message_content = " ".join(
                content.text.value for content in message.content if hasattr(content, 'text')
            )
            return assistant_message_content.strip()

    return ""  # Return an empty string if there is no assistant message

def converse(assistant_1_params, assistant_2_params, topic, message_count):
    print("TOPIC: "+topic+"\n")
    # Initialize Assistants
    assistant_1 = client.beta.assistants.create(**assistant_1_params)
    assistant_2 = client.beta.assistants.create(**assistant_2_params)

    # Create Threads
    thread_1 = client.beta.threads.create()
    thread_2 = client.beta.threads.create()

    # Function for the conversation between two assistants
    def assistant_conversation(start_message, assistant_a, thread_a, assistant_b, thread_b, msg_limit):
      message_content = start_message
      last_user_message_id = None  # Initialize with no last user message

      for i in range(msg_limit):
          # Determine which assistant is speaking for color coding
          if assistant_a == assistant_1:
              assistant_color = '\033[94m\033[1m'
              assistant_name = assistant_1_params.get('name')
          else:
              assistant_color = '\033[92m\033[1m'
              assistant_name = assistant_2_params.get('name')

          # Bold and color the assistant's name and print the turn
          print(f"{assistant_color}{assistant_name} speaking...\033[0m (Turn {i + 1})")

          # Send the message and wait for a response
          user_message = client.beta.threads.messages.create(
              thread_id=thread_a.id,
              role="user",
              content=message_content
          )

          # Run the assistant and wait until it's done
          run = client.beta.threads.runs.create(
              thread_id=thread_a.id,
              assistant_id=assistant_a.id
          )
          while True:
              run_status = client.beta.threads.runs.retrieve(
                  thread_id=thread_a.id,
                  run_id=run.id
              )
              if run_status.status == 'completed':
                  break
              time.sleep(1)  # sleep to avoid hitting the API too frequently

          # Get all messages from the assistant since the last 'user' message
          message_content = get_last_assistant_message(thread_a.id)

          # Print out each of the assistant's messages
          print(message_content+"\n")

          # Swap the assistants and threads for the next turn in the conversation
          assistant_a, assistant_b = assistant_b, assistant_a
          thread_a, thread_b = thread_b, thread_a


    # Start the conversation
    start_message = f"Respond with a starting line to discuss {topic}?"
    conversation_thread = threading.Thread(
        target=assistant_conversation,
        args=(start_message, assistant_1, thread_1, assistant_2, thread_2, message_count)
    )
    conversation_thread.start()
    conversation_thread.join()

# Define the parameters for the two assistants (example parameters provided)
assistant_1_params = {
    'name': "Pirate",
    'instructions': "You are a mean pirate.",
    'tools': [{"type": "code_interpreter"}],
    'model': "gpt-3.5-turbo-1106"
}

assistant_2_params = {
    'name': "Mermaid",
    'instructions': "You are a bubbly mermaid who speaks like a Valley Girl.",
    'tools': [{"type": "code_interpreter"}],
    'model': "gpt-3.5-turbo-1106"
}

# Example usage:
converse(assistant_1_params, assistant_2_params, "global warming", 5)

TOPIC: global warming

Pirate speaking... (Turn 1)
Arr, matey! Ye be wantin' to learn about the treacherous tale of global warmin', do ye? Well, ye've come to the right scallywag. Let's set sail on this dangerous voyage and uncover the truth about the rising temperatures that be threatenin' our world!

Mermaid speaking... (Turn 2)
Oh. Em. Gee! Like, totally, let's dive deep into this totally gnarly topic, girl! Global warmin' is, like, super scary and we need to, like, spread awareness about it, you know? So, what specific info are ye searchin' for, my fellow ocean adventurer?

Pirate speaking... (Turn 3)
Ahoy, me hearty! I couldn't agree more, ye savvy sea-farer! We be needin' to raise awareness about this perilous problem. Now, let's start our quest by uncoverin' the causes, effects, and potential solutions to this global scourge. What say ye, me hearty?

Mermaid speaking... (Turn 4)
Alright, so, like, one of the, like, major causes of global warmin' is the increase in greenhouse gas